# 📱 Mobile Application Data Generator - PySpark

Generate large-scale mobile application data and save to Delta Lake for performance testing.

## Features:
- **Configurable record count** - from 20M to 200M+ records
- **Auto-scaling partitions** - adjust based on record count
- **Distributed processing** - PySpark DataFrame for large datasets
- **State-based partitioning** - partitioned by `fields.state` for optimal performance
- **Unique _id generation** - prevents duplicates
- **Nested JSON structure** - ready for database insertion
- **Delta Lake format** - ACID transactions and efficient queries

## Configuration:
Change `NUM_RECORDS` in the configuration cell:
- **20M records**: `NUM_RECORDS = 20000000`
- **50M records**: `NUM_RECORDS = 50000000`
- **100M records**: `NUM_RECORDS = 100000000`
- **200M records**: `NUM_RECORDS = 200000000`

In [ ]:
# Configuration - Adjust these parameters as needed
NUM_RECORDS = 200000000
NUM_PARTITIONS = 1000
DELTA_TABLE_PATH = "abfss://datacare@onelake.dfs.fabric.microsoft.com/mobileapps.Lakehouse/Tables/att/mobile_applications_partitioned"

In [ ]:
# Import required libraries
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.sql.window import Window
import random
import uuid
from datetime import datetime, timedelta

print("✅ Libraries imported successfully")
print(f"📊 Using existing Spark session")

In [ ]:
# Auto-adjust partitions based on record count
if NUM_RECORDS <= 20000000:
    RECOMMENDED_PARTITIONS = 200
elif NUM_RECORDS <= 50000000:
    RECOMMENDED_PARTITIONS = 500
elif NUM_RECORDS <= 100000000:
    RECOMMENDED_PARTITIONS = 800
else:
    RECOMMENDED_PARTITIONS = 1000

NUM_PARTITIONS = RECOMMENDED_PARTITIONS

print(f"🎯 Generating {NUM_RECORDS:,} records")
print(f"⚡ Using {NUM_PARTITIONS} partitions")

In [ ]:
# Data configuration - US states, device types, and sample data
ALL_US_STATES = ["AL", "AK", "AZ", "AR", "CA", "CO", "CT", "DE", "FL", "GA", "HI", "ID", "IL", "IN", "IA", "KS", 
                 "KY", "LA", "ME", "MD", "MA", "MI", "MN", "MS", "MO", "MT", "NE", "NV", "NH", "NJ", "NM", "NY", 
                 "NC", "ND", "OH", "OK", "OR", "PA", "RI", "SC", "SD", "TN", "TX", "UT", "VT", "VA", "WA", "WV", "WI", "WY", "DC"]

DEVICE_MANUFACTURERS = ["Apple", "Samsung", "Google", "OnePlus", "Motorola"]
DEVICE_MODELS = {
    "Apple": ["iPhone 15 Pro", "iPhone 15", "iPhone 14"], "Samsung": ["Galaxy S24", "Galaxy S23", "Galaxy A54"],
    "Google": ["Pixel 8 Pro", "Pixel 8", "Pixel 7a"], "OnePlus": ["OnePlus 12", "OnePlus 11", "OnePlus Nord"],
    "Motorola": ["Moto G Power", "Moto Edge", "Razr+"]
}

OPERATING_SYSTEMS = [
    {"os": "iOS", "versions": ["17.1", "17.0", "16.7"]},
    {"os": "Android", "versions": ["14.0", "13.0", "12.0"]}
]

APP_CATEGORIES = ["Social Media", "Gaming", "Productivity", "Entertainment", "Shopping", "Finance", "Travel"]
MOBILE_CARRIERS = ["Verizon", "AT&T", "T-Mobile", "Sprint", "US Cellular", "Cricket"]
FIRST_NAMES = ["James", "Mary", "John", "Patricia", "Robert", "Jennifer", "Michael", "Linda"]
LAST_NAMES = ["Smith", "Johnson", "Williams", "Brown", "Jones", "Garcia", "Miller", "Davis"]
CITIES = ["New York", "Los Angeles", "Chicago", "Houston", "Phoenix", "Philadelphia", "Dallas", "Austin"]

print(f"✅ Data config: {len(ALL_US_STATES)} states, {len(DEVICE_MANUFACTURERS)} manufacturers")

In [ ]:
# UDFs for random data generation
@udf(returnType=StringType())
def generate_id(partition_id, row_num):
    import time, uuid
    timestamp = int(time.time() * 1000)
    unique_part = str(uuid.uuid4()).replace('-', '')[:8]
    return f"app_{timestamp}_{partition_id:03d}_{row_num:08d}_{unique_part}"

@udf(returnType=StringType())
def random_string(length):
    import random, string
    return ''.join(random.choices(string.ascii_letters + string.digits, k=length))

@udf(returnType=StringType())
def random_datetime():
    import random
    from datetime import datetime, timedelta
    base_date = datetime(2024, 1, 1)
    random_days = random.randint(0, 365)
    result_date = base_date + timedelta(days=random_days)
    return result_date.isoformat() + "Z"

@udf(returnType=StringType())
def random_choice(choices):
    import random
    return random.choice(choices)

@udf(returnType=IntegerType())
def random_int(min_val, max_val):
    import random
    return random.randint(min_val, max_val)

@udf(returnType=BooleanType())
def random_bool():
    import random
    return random.choice([True, False])

print("✅ UDFs defined")

In [ ]:
# Generate base DataFrame
print(f"🚀 Generating {NUM_RECORDS:,} records...")

base_df = spark.range(0, NUM_RECORDS, numPartitions=NUM_PARTITIONS) \
    .withColumn("partition_id", spark_partition_id()) \
    .withColumn("row_num", row_number().over(Window.partitionBy("partition_id").orderBy("id")))

print(f"✅ Base DataFrame created")

In [ ]:
# Generate mobile app data
print("📱 Creating mobile app records...")

mobile_app_df = base_df \
    .withColumn("_id", generate_id(col("partition_id"), col("row_num"))) \
    .withColumn("created_dt", random_datetime()) \
    .withColumn("applicationTimeStamp", col("created_dt")) \
    .withColumn("createdDateTime", col("created_dt")) \
    .withColumn("lastUpdateDateTime", random_datetime()) \
    .withColumn("serviceId", random_string(lit(6))) \
    .withColumn("version", random_string(lit(32))) \
    .withColumn("_class", random_string(lit(35))) \
    .withColumn("temp_firstName", random_choice(array([lit(name) for name in FIRST_NAMES]))) \
    .withColumn("temp_lastName", random_choice(array([lit(name) for name in LAST_NAMES]))) \
    .withColumn("temp_state", random_choice(array([lit(state) for state in ALL_US_STATES]))) \
    .withColumn("temp_zip", lpad(random_int(lit(10000), lit(99999)).cast("string"), 5, "0")) \
    .withColumn("temp_city", random_choice(array([lit(city) for city in CITIES]))) \
    .withColumn("temp_manufacturer", random_choice(array([lit(mfg) for mfg in DEVICE_MANUFACTURERS]))) \
    .withColumn("temp_deviceModel", 
        when(col("temp_manufacturer") == "Apple", 
             random_choice(array([lit(model) for model in DEVICE_MODELS["Apple"]]))
        ).when(col("temp_manufacturer") == "Samsung", 
               random_choice(array([lit(model) for model in DEVICE_MODELS["Samsung"]]))
        ).when(col("temp_manufacturer") == "Google", 
               random_choice(array([lit(model) for model in DEVICE_MODELS["Google"]]))
        ).when(col("temp_manufacturer") == "OnePlus", 
               random_choice(array([lit(model) for model in DEVICE_MODELS["OnePlus"]]))
        ).otherwise(
               random_choice(array([lit(model) for model in DEVICE_MODELS["Motorola"]]))
        )
    ) \
    .withColumn("temp_os", when(col("temp_manufacturer") == "Apple", lit("iOS")).otherwise(lit("Android"))) \
    .withColumn("temp_osVersion",
        when(col("temp_os") == "iOS", 
             random_choice(array([lit(v) for v in OPERATING_SYSTEMS[0]["versions"]]))
        ).otherwise(
             random_choice(array([lit(v) for v in OPERATING_SYSTEMS[1]["versions"]]))
        )
    ) \
    .withColumn("fields", struct(
        # User Information
        col("temp_firstName").alias("firstName"),
        col("temp_lastName").alias("lastName"),
        concat(col("temp_firstName"), lit(" "), col("temp_lastName")).alias("fullName"),
        col("temp_firstName").alias("conFirstname"),
        col("temp_lastName").alias("conLastname"),
        
        # Geographic Information (PARTITION KEY)
        col("temp_state").alias("state"),
        col("temp_state").alias("conState"),
        col("temp_zip").alias("zip"),
        col("temp_zip").alias("conZipCode"),
        col("temp_city").alias("city"),
        col("temp_city").alias("conCity"),
        lit("US").alias("conCountry"),
        
        # Device Information
        col("temp_manufacturer").alias("deviceManufacturer"),
        col("temp_deviceModel").alias("deviceModel"),
        col("temp_os").alias("operatingSystem"),
        col("temp_osVersion").alias("osVersion"),
        
        
        # Mobile and Network Information
        random_choice(array([lit(carrier) for carrier in MOBILE_CARRIERS])).alias("mobileCarrier"),
        random_choice(array([lit("WiFi"), lit("4G"), lit("5G"), lit("LTE")])).alias("connectionType"),
        random_string(lit(16)).alias("deviceId"),
        
        # Application Information
        random_choice(array([lit(cat) for cat in APP_CATEGORIES])).alias("appCategory"),
        random_string(lit(20)).alias("sessionId"),
        random_int(lit(30), lit(7200)).alias("sessionDuration"),
        random_int(lit(10), lit(100)).alias("batteryLevel"),
        random_int(lit(20), lit(500)).alias("networkLatency"),
        
        # Credit and Financial Information
        random_int(lit(300), lit(850)).alias("conModelScore"),
        random_string(lit(2)).alias("conApplicationStatus"),
        random_choice(array([lit("APPROVED"), lit("DECLINED"), lit("PENDING")])).alias("conCurrentDecision"),
        random_int(lit(10), lit(99)).cast("string").alias("affiliateId"),
        random_string(lit(2)).alias("conAffiliateId"),
        
        # Additional fields
        random_string(lit(9)).alias("conSSN"),
        random_string(lit(3)).alias("conApplicationType"),
        random_bool().alias("isPkMatch"),
        random_datetime().alias("effectiveDateTime"),
        random_datetime().alias("timestamp")
    )) \
    .withColumn("state", col("temp_state")) \
    .drop("temp_firstName", "temp_lastName", "temp_state", "temp_zip", "temp_city", 
          "temp_manufacturer", "temp_deviceModel", "temp_os", "temp_osVersion") \
    .select("_id", "applicationTimeStamp", "createdDateTime", "lastUpdateDateTime", 
            "serviceId", "version", "_class", "fields", "state") \
    .repartition(col("state"))  # Repartition by state for optimal write performance

print("✅ Mobile app DataFrame generated")

In [ ]:
# Show sample data
mobile_app_df.select("_id", "fields.firstName", "fields.lastName", "fields.state", 
                     "fields.deviceManufacturer", "fields.appCategory").show(5, truncate=False)

print("✅ Sample data displayed")

In [ ]:
# Write to Delta Lake
print("💾 Writing to Delta Lake...")

mobile_app_df.write \
    .format("delta") \
    .mode("overwrite") \
    .partitionBy("state") \
    .option("path", DELTA_TABLE_PATH) \
    .save()

print(f"✅ Data written to: {DELTA_TABLE_PATH}")
print(f"🗂️  Partitioned by state")

In [ ]:
# Verify data
delta_df = spark.read.format("delta").load(DELTA_TABLE_PATH)
print(f"✅ Verified: {delta_df.count():,} records")

# Show distribution
delta_df.groupBy("state").count().orderBy(desc("count")).show(10)
delta_df.groupBy("fields.deviceManufacturer").count().show()

## 🎯 Summary

### ✅ Generated Data:
- **200 million records** - Configurable from 20M to 200M+
- **Auto-scaling partitions** - Adjust based on record count
- **State-based partitioning** - Optimized for geographic queries
- **Nested JSON structure** - Ready for database insertion
- **Mobile application data** - Devices, carriers, apps, users

### 🗂️ Delta Lake Output:
- **Format**: Partitioned Delta table by `fields.state`
- **Location**: Fabric Lakehouse
- **Features**: ACID transactions, time travel, schema evolution
- **Performance**: Optimized for state-based queries and aggregations

### 🔧 Data Schema:
- **Unique _id**: `app_{timestamp}_{partition}_{row}_{uuid}`
- **Partition key**: `fields.state` (50 US states + DC)
- **Fields**: User info, device details, geographic data, financial attributes

### 🚀 Next Steps:
1. Verify data using the cells above
2. Use **`02_mongodb_data_insertion_pyspark.ipynb`** to insert into Azure DocumentDB
3. Run **`03_create_mongodb_indexes.py`** to create indexes
4. Execute **`04_run_mongodb_queries.py`** for performance testing